# campAIgnR v01


#### Import

In [ ]:
from utils import *

### 1 Concept Prompt and Execution

In [ ]:
#campaign_ideas = generate_campaign_concept(15)
#campaign_ideas

In [ ]:
#campaign_ideas[11]

### 2 Basic Plan Generation

In [ ]:
#campaign_concept  = campaign_ideas[11]



campaign_concept =  'Issue: Hydration Habits on Campus – A Thirst for Change. Need: Many college students are under-hydrated, unaware of how much water they need, and unsure how to build sustainable hydration habits. Goal: Educate students about the importance of proper hydration, provide easy and practical strategies, and motivate lasting behavior change for better health and energy.'
    
basic_campaign_plan = generate_basic_campaign_plan(campaign_concept)

#for i in range(len(basic_campaign_plan)):
#    print(i)
#    print(basic_campaign_plan[i][:50])
#    print('----')

In [ ]:
for i in range(len(basic_campaign_plan)):
    print(i)
    print(basic_campaign_plan[i][:50])
    print('----')

### 3 Title Extraction and Cover Image

In [ ]:
campaign_title_prompt = "Extract and return only the campaign title from the following description" +\
                        "and then add '- A campAIgnR project'. Just return the title and appendix, no other symbols or text, especially no colons, question or exclamaion marks, or anything like that."+\
                        "Here comes the description -> " + basic_campaign_plan[0] + ". "

campaign_title = generate_simple(campaign_title_prompt)
image_title_path = generate_campaign_cover(campaign_title)

In [ ]:
basic_campaign_plan[1]

### 6 Generate Main Story Line

In [ ]:
revised_sections = []
n_sections = len(basic_campaign_plan)
print('Title: ')
print(basic_campaign_plan[0])

for current_section in np.arange(1, n_sections):
    print('------------ SECTION -----------')
    print(current_section +1 )
    section_writeout_prompt =  "You are writing a campaign plan for a public health campaign" +\
                               "You are given a high-level concept description and then a specific section that you will write out."+\
                               "Overall concept: -> " + campaign_concept + ". This was the overall concept. The campaign title is " +\
                               "Title: -> " + campaign_title + ".  The campaign has the following sections: Title, Introduction, Goals " +\
                               "Formative Research (Strucured into Situation Analysis,Audience Analysis, and Previous Communication Evaluation, "+\
                               "Theory and Messages, Exposure and Channels, Evaluation, and Appendix (budget, timeline, personnel etc). "+\
                               "Your job now is to write out one specific section. In particular, you are to expand this "+\
                               "section: " + basic_campaign_plan[current_section] + ". This was the section. Now your job:"+\
                               "Write out this section into a compelling plan. "+\
                               "You can use 600 words or even more, though some sections (like introduction) could also be shorter"+\
                               " Make everything well-connected, documented, backed by facts (true ones, not made up)." +\
                               "Use short and well-crafed sentences." +\
                               "Make sure the section is logical and well-connected to the other sections, which you may reference. " +\
                               "It is also engaging, written in a positive tone."+\
                               "You remove all symbols, like '#' , or emojis, or other weird things like '[]' or '{}' etc. Just return very clean and plain text." 
                               
    result = generate_simple(section_writeout_prompt)
    print(result)   
    
        
    revised_sections.append(result)



In [ ]:
def flatten_list(nested_list):
    """Recursively flatten a nested list and prepend an index."""
    index = 1  # Start index at 1
    for item in nested_list:
        if isinstance(item, list):
            # Yield the list header
            yield f"\n\nScene #{index}\n\n"  # Add the list header
            yield from flatten_list(item)  # Recurse into the sublist
            index += 1  # Increment the index for each sublist
        else:
            yield item  # If it's not a list, yield the item

cleaned_text = []
for line in flatten_list(revised_sections):
    cleaned_line = line.lstrip('``')  # Remove leading ```
    cleaned_line = line.lstrip('``')  # Remove leading ```
    cleaned_line = cleaned_line.replace('plaintext', '')  # Remove 'plaintext'
    cleaned_line = cleaned_line.rstrip(',')  # Remove trailing comma
    cleaned_line = cleaned_line.replace('[', '').replace(']', '')  # Remove brackets 
    cleaned_text.append(cleaned_line.strip())  # Remove any leading/trailing whitespace

def sanitize_text(text):
    # Replace special characters with simpler alternatives
    replacements = {
        '\u2014': '--',  # Replace em dash with double dash
        '\u2013': '-',   # Replace en dash with a single dash
        '\u201c': '"',   # Replace left double quote
        '\u201d': '"',   # Replace right double quote
        '\u2018': "'",   # Replace left single quote
        '\u2019': "'",   # Replace right single quote
        '\u2026': ".",
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

# Sanitize text before adding to PDF
cleaned_text = [sanitize_text(line) for line in cleaned_text]

text_file_path = "./00_ingredients/script.txt"
with open(text_file_path, mode='w', encoding='utf-8') as file:
    file.write(campaign_title + '\n\n')  
    for line in cleaned_text:
        file.write(line + '\n')  
print(f"List written to {text_file_path}")

In [ ]:
revised_sections[0]

In [ ]:
def generate_sections(sections_list):
    sections = []
    for section in sections_list:
        # Split title and content based on the first newline
        title, content = section.split("\n\n", 1)
        sections.append((title, content))  # Add the title and content as a tuple
    return sections

# Generate sections dynamically
sections = generate_sections(revised_sections)
sections

In [ ]:
from jinja2 import Environment, FileSystemLoader
from datetime import datetime
import pdfkit


image_path = os.path.abspath(image_title_path)
image_url = f"file://{image_path}"

# Define the report data, including the image URL or path
report_data = {
    'report_title': campaign_title,
    'author_name': "gpt-4o-mini - directed by NAME HERE",
    'title_image_url': image_url,  # Local path or URL
    'sections':  sections,
    'generation_date': datetime.now().strftime('%Y-%m-%d')
}

# Render the template
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("./00_ingredients/report_template.html")
html_content = template.render(report_data)

# Save the rendered HTML to a file
html_output_path = "./01_output/" + campaign_title + '/' + campaign_title +".html"
pdf_output_path = "./01_output/" + campaign_title + '/' + campaign_title  +".pdf"

with open(html_output_path , "w") as file:
    file.write(html_content)


options = {'enable-local-file-access': True}

# Convert HTML to PDF with options
pdfkit.from_file(html_output_path, pdf_output_path, options=options)

